<a href="https://colab.research.google.com/github/iremnym/My-deeplearning-projects/blob/main/image-text-with-blip-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y torch torchvision torchaudio transformers accelerate peft datasets pillow tqdm pandas numpy tensorflow


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
Found existing installation: pillow 11.2.1
Uninstalling pillow-11.2.1:
  Successfully uninstalled pillow-11.2.1
Found

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!pip install --quiet \
  numpy==1.25.2 \
  pandas==2.2.2 \
  torch==2.0.1 \
  torchvision==0.15.2 \
  torchaudio==2.0.2 \
  transformers==4.30.2 \
  accelerate==0.20.3 \
  peft==0.4.0 \
  datasets==2.13.0 \
  pillow==10.0.0 \
  tqdm==4.65.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8

In [17]:
from transformers import BlipProcessor, BlipForConditionalGeneration, Trainer, TrainingArguments, default_data_collator
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import torch
import os
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cuda


In [18]:

model_id = "Salesforce/blip-image-captioning-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained(model_id)

model = BlipForConditionalGeneration.from_pretrained(model_id)
model.to(device)



BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [19]:
class OBSSDataset(Dataset):
    def __init__(self, dataframe, image_dir, processor):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.processor = processor
        self.target_dtype = torch.float16 if torch.cuda.is_available() and model.dtype == torch.float16 else torch.float32

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_dir, f"{row['image_id']}.jpg")
        image = Image.open(image_path).convert("RGB")
        caption = row["caption"]

        inputs = self.processor(
            images=image,
            text=caption,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=128
        )

        # Batch dimension'ı sıkıştır
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}

        # labels ayarı
        inputs["labels"] = inputs["input_ids"].clone()
        inputs["labels"][inputs["attention_mask"] == 0] = -100
        inputs["pixel_values"] = inputs["pixel_values"].to(self.target_dtype)

        return inputs


In [21]:
train_df = pd.read_csv("/content/drive/MyDrive/obss-competition-files/train.csv")
train_img_dir = ("/content/drive/MyDrive/obss-competition-files/train/train")
dataset = OBSSDataset(train_df, train_img_dir, processor)


In [25]:
from transformers import TrainerCallback

class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            print(f"[Trainer] Step: {state.global_step}, Loss: {logs['loss']}")


In [35]:
training_args = TrainingArguments(
    output_dir="./blip_finetuned",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    save_strategy="no",
    logging_steps=10,
    remove_unused_columns=False,
    report_to="none",
    seed=42,
    fp16=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=default_data_collator,
    tokenizer=processor,  # uyarı veriyor ama çalışıyor
    callbacks=[LoggingCallback()]
)


trainer.train()

<ipython-input-35-aba960125b98>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.971000
20,1.898500
30,1.798300
40,1.767900
50,1.760300
60,1.931400
70,2.177000
80,2.199000
90,2.123700
100,2.127100


[Trainer] Step: 10, Loss: 1.971
[Trainer] Step: 20, Loss: 1.8985
[Trainer] Step: 30, Loss: 1.7983
[Trainer] Step: 40, Loss: 1.7679
[Trainer] Step: 50, Loss: 1.7603
[Trainer] Step: 60, Loss: 1.9314
[Trainer] Step: 70, Loss: 2.177
[Trainer] Step: 80, Loss: 2.199
[Trainer] Step: 90, Loss: 2.1237
[Trainer] Step: 100, Loss: 2.1271
[Trainer] Step: 110, Loss: 2.108
[Trainer] Step: 120, Loss: 2.0551
[Trainer] Step: 130, Loss: 2.0843
[Trainer] Step: 140, Loss: 2.1516
[Trainer] Step: 150, Loss: 2.0374
[Trainer] Step: 160, Loss: 2.0105
[Trainer] Step: 170, Loss: 2.0679
[Trainer] Step: 180, Loss: 2.009
[Trainer] Step: 190, Loss: 1.9697
[Trainer] Step: 200, Loss: 2.0685
[Trainer] Step: 210, Loss: 2.0776
[Trainer] Step: 220, Loss: 1.9779
[Trainer] Step: 230, Loss: 2.001
[Trainer] Step: 240, Loss: 1.9695
[Trainer] Step: 250, Loss: 1.9091
[Trainer] Step: 260, Loss: 2.0379
[Trainer] Step: 270, Loss: 1.9216
[Trainer] Step: 280, Loss: 1.9454
[Trainer] Step: 290, Loss: 1.9326
[Trainer] Step: 300, Loss: 1.

TrainOutput(global_step=2004, training_loss=1.3660268252957128, metrics={'train_runtime': 1408.5247, 'train_samples_per_second': 45.509, 'train_steps_per_second': 1.423, 'total_flos': 3.803904953127626e+19, 'train_loss': 1.3660268252957128, 'epoch': 3.0})

In [36]:

trainer.save_model("finetuned_blip")
processor.save_pretrained("finetuned_blip")


[]

In [37]:
!cp -r finetuned_blip /content/drive/MyDrive/


In [38]:
!zip -r finetuned_blip.zip finetuned_blip
from google.colab import files
files.download("finetuned_blip.zip")


  adding: finetuned_blip/ (stored 0%)
  adding: finetuned_blip/model.safetensors (deflated 7%)
  adding: finetuned_blip/preprocessor_config.json (deflated 48%)
  adding: finetuned_blip/generation_config.json (deflated 28%)
  adding: finetuned_blip/vocab.txt (deflated 53%)
  adding: finetuned_blip/tokenizer_config.json (deflated 74%)
  adding: finetuned_blip/config.json (deflated 54%)
  adding: finetuned_blip/tokenizer.json (deflated 71%)
  adding: finetuned_blip/special_tokens_map.json (deflated 80%)
  adding: finetuned_blip/training_args.bin (deflated 52%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm


device = "cuda" if torch.cuda.is_available() else "cpu"


model_path = "./finetuned_blip"
processor = BlipProcessor.from_pretrained(model_path)
model = BlipForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)


test_df = pd.read_csv("/content/drive/MyDrive/obss-competition-files/test.csv")
test_img_dir = "/content/drive/MyDrive/obss-competition-files/test/test"

results = []


prompt = (
    "Describe the image with detailed, vivid, and specific language. "
    "Focus on the actions, scene context, objects, colors, and interactions. "
    "Avoid generic captions. Write like a human observer."
)


for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = os.path.join(test_img_dir, f"{row['image_id']}.jpg")
    image = Image.open(image_path).convert("RGB")

    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=60,
        num_beams=7,
        no_repeat_ngram_size=3,
        repetition_penalty=1.2,
        length_penalty=1.1,
        early_stopping=True
    )

    caption = processor.decode(generated_ids[0], skip_special_tokens=True).strip()
    results.append({"image_id": row["image_id"], "caption": caption})

# Caption sonuçlarını kaydet
submission_df = pd.DataFrame(results)
submission_df.to_csv("submission.csv", index=False)


100%|██████████| 3771/3771 [16:04<00:00,  3.91it/s]
